In [1]:
import numpy as np
import os

In [2]:
spectrogram = os.listdir('spectrogram')
len(spectrogram)

17399

In [3]:
X, Y = [], []
for spec in spectrogram:
    Y.append(' '.join([i for i in os.path.splitext(spec)[0].split('-') if not i.isdigit()]))
    X.append(np.load('spectrogram/' + spec))

In [4]:
import tensorflow as tf
from tqdm import tqdm

In [ ]:
chars = list(set([c for target in Y for c in target]))
num_classes = len(chars) + 2

idx2char = {idx + 1: char for idx, char in enumerate(chars)}
idx2char[0] = '<PAD>'
char2idx = {char: idx for idx, char in idx2char.items()}

Y = [[char2idx[c] for c in target] for target in Y]

In [ ]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

def sparse_tuple_from(sequences, dtype=np.int32):
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n] * len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1] + 1], dtype=np.int64)

    return indices, values, shape

In [ ]:
tf.__version__

'1.10.0'

In [ ]:
def pad_causal(x, size, rate):
    pad_len = (size - 1) * rate
    return tf.pad(x, [[0, 0], [pad_len, 0], [0, 0]])

def pad_second_dim(x, desired_size):
    padding = tf.tile([[0]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1]], 0))
    return tf.concat([x, padding], 1)

class Model:
    def __init__(
        self,
        num_layers,
        size_layers,
        learning_rate,
        num_features,
        num_blocks = 3,
        block_size = 128,
        dropout = 1.0,
    ):
        self.X = tf.placeholder(tf.float32, [None, None, num_features])
        self.Y = tf.sparse_placeholder(tf.int32)
        seq_lens = tf.count_nonzero(
            tf.reduce_sum(self.X, -1), 1, dtype = tf.int32
        ) + 10
        filled = tf.fill(tf.shape(seq_lens), tf.shape(self.X)[1])
        seq_lens = tf.where(seq_lens > tf.shape(self.X)[1], filled, seq_lens)
        self.label = tf.placeholder(tf.int32, [None, None])
        self.Y_seq_len = tf.placeholder(tf.int32, [None])

        def residual_block(x, size, rate, block):
            with tf.variable_scope('block_%d_%d' % (block, rate), reuse = False):
                conv_filter = tf.layers.conv1d(
                    x,
                    x.shape[2] // 4,
                    kernel_size = size,
                    strides = 1,
                    padding = 'same',
                    dilation_rate = rate,
                    activation = tf.nn.tanh,
                )
                conv_gate = tf.layers.conv1d(
                    x,
                    x.shape[2] // 4,
                    kernel_size = size,
                    strides = 1,
                    padding = 'same',
                    dilation_rate = rate,
                    activation = tf.nn.sigmoid,
                )
                out = tf.multiply(conv_filter, conv_gate)
                out = tf.layers.conv1d(
                    out,
                    block_size,
                    kernel_size = 1,
                    strides = 1,
                    padding = 'same',
                    activation = tf.nn.tanh,
                )
                return tf.add(x, out), out
        forward = tf.layers.conv1d(self.X, block_size, kernel_size = 1, strides = 1, padding = 'SAME')
        zeros = tf.zeros_like(forward)
        for i in range(num_blocks):
            for r in [1, 2, 4, 8, 16]:
                forward, s = residual_block(forward, size=7, rate=r, block=i)
                zeros = tf.add(zeros,s)
        logits = tf.layers.conv1d(zeros, num_classes, kernel_size = 1, strides = 1, padding = 'SAME')
        time_major = tf.transpose(logits, [1, 0, 2])
        decoded, log_prob = tf.nn.ctc_beam_search_decoder(time_major, seq_lens)
        decoded = tf.to_int32(decoded[0])
        self.preds = tf.sparse_tensor_to_dense(decoded)
        self.cost = tf.reduce_mean(
            tf.nn.ctc_loss(
                self.Y,
                time_major,
                seq_lens
            )
        )
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        
        preds = self.preds[:, :tf.reduce_max(self.Y_seq_len)]
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        preds = pad_second_dim(preds, tf.reduce_max(self.Y_seq_len))
        y_t = tf.cast(preds, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.label, masks)
        self.mask_label = mask_label
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

size_layers = 256
learning_rate = 1e-4
num_layers = 2
batch_size = 32
epoch = 50

model = Model(num_layers, size_layers, learning_rate, X[0].shape[1])
sess.run(tf.global_variables_initializer())

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(X, Y)
del X

In [ ]:
for e in range(epoch):
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    total_cost, total_accuracy = 0, 0
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, len(train_X))]
        batch_x = tf.keras.preprocessing.sequence.pad_sequences(batch_x, dtype = 'float32', padding = 'post')
        y = train_Y[i : min(i + batch_size, len(train_X))]
        batch_y = sparse_tuple_from(y)
        batch_label, batch_len = pad_sentence_batch(y, 0)
        _, cost, accuracy = sess.run(
            [model.optimizer, model.cost, model.accuracy],
            feed_dict = {model.X: batch_x, model.Y: batch_y, 
                         model.label: batch_label, model.Y_seq_len: batch_len},
        )
        total_cost += cost
        total_accuracy += accuracy
        pbar.set_postfix(cost = cost, accuracy = accuracy)
    total_cost /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    print('epoch %d, training average cost %f, training average accuracy %f'%(e + 1, total_cost, total_accuracy))
    
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'testing minibatch loop')
    total_cost, total_accuracy = 0, 0
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, len(test_X))]
        batch_x = tf.keras.preprocessing.sequence.pad_sequences(batch_x, dtype = 'float32', padding = 'post')
        y = test_Y[i : min(i + batch_size, len(test_X))]
        batch_y = sparse_tuple_from(y)
        batch_label, batch_len = pad_sentence_batch(y, 0)
        _, cost, accuracy = sess.run(
            [model.optimizer, model.cost, model.accuracy],
            feed_dict = {model.X: batch_x, model.Y: batch_y, 
                         model.label: batch_label, model.Y_seq_len: batch_len},
        )
        total_cost += cost
        total_accuracy += accuracy
        pbar.set_postfix(cost = cost, accuracy = accuracy)
        
    total_cost /= (len(test_X) / batch_size)
    total_accuracy /= (len(test_X) / batch_size)
    print('epoch %d, testing average cost %f, testing average accuracy %f'%(e + 1, total_cost, total_accuracy))

testing minibatch loop:   0%|          | 0/136 [00:00<?, ?it/s]

epoch 1, training average cost 63.704233, training average accuracy 0.131253


minibatch loop:   0%|          | 0/408 [00:00<?, ?it/s]

epoch 1, testing average cost 43.260290, testing average accuracy 0.290187


testing minibatch loop:   0%|          | 0/136 [00:00<?, ?it/s]

...

testing minibatch loop:   0%|          | 0/136 [00:00<?, ?it/s]

epoch 25, training average cost 9.449029, training average accuracy 0.666897


minibatch loop:   0%|          | 0/408 [00:00<?, ?it/s]

epoch 25, testing average cost 9.337638, testing average accuracy 0.662240


minibatch loop:  89%|████████▉ | 364/408 [04:04<00:28,  1.52it/s, accuracy=0.602, cost=9.35]

In [13]:
import random

random_index = random.randint(0, len(test_X) - 1)
batch_x = test_X[random_index : random_index + 1]
print(
    'real:',
    ''.join(
        [idx2char[no] for no in test_Y[random_index : random_index + 1][0]]
    ),
)
batch_y = sparse_tuple_from(test_Y[random_index : random_index + 1])
pred = sess.run(model.preds, feed_dict = {model.X: batch_x})[0]
print('predicted:', ''.join([idx2char[no] for no in pred]))

real: sebut perkataan man pengaktifan
predicted: sebut perkatan man aengaki
